<a href="https://colab.research.google.com/github/DevanMayer/EE-480-Assignments/blob/main/CA6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **EE 480 - Coding Assignment #6: Optimal Power Flow & Market Simulation**

**Instructor:** Dr. Saeed Manshadi

**Due Date:** Nov. 17, 2025

### **Instructions**

This assignment is the culmination of our work on Optimal Power Flow (OPF) and its application in electricity markets. You will compare different OPF formulations and participate in a simulated market to develop a profit-maximizing bidding strategy. All solutions must be calculated using **Julia** and the `PowerModels.jl` package.

**Working in Google Colab:**
*   This notebook is designed to be completed in Google Colab.
*   **CRITICAL FIRST STEP:** Ensure the Julia kernel is active. Run the setup cell below.

---
### **Required Packages (Run This Cell First!)**

In [5]:
# This cell adds the necessary packages for this assignment.
using Pkg
Pkg.add("PowerModels")
Pkg.add("Ipopt")
Pkg.add("JSON")
Pkg.add("LinearAlgebra")
Pkg.add("Printf") # For formatted printing
Pkg.add("PowerModels")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


---
### **Problem 1: Comparing DC OPF and AC OPF**

The DC OPF is a linear approximation, while the AC OPF is a full, nonlinear model. In this problem, you will solve both for the standard IEEE 14-bus system and analyze the differences in the results.

**Guidance:**
*   Load the `case14.m` file, which includes default generator cost data.
*   Run the `solve_dc_opf` function to get the DC solution.
*   Run the `solve_ac_opf` function to get the AC solution.
*   Present your results in a clean, comparative table.

In [ ]:
# --- Problem 1 Solution ---
using PowerModels, Ipopt, Printf, LinearAlgebra, JSON

# --- Step 1: Load the test case ---
case_file_path = joinpath(dirname(pathof(PowerModels)), "..", "test/data/matpower/case14.m")
case_14bus = PowerModels.parse_file(case_file_path)

# --- Step 2: Solve the DC OPF ---
result_dc = solve_dc_opf(case_14bus, DCMPowerModel, Ipopt.Optimizer)


# --- Step 3: Solve the AC OPF ---
result_ac = solve_ac_opf(case_14bus, ACPPowerModel, Ipopt.Optimizer)


# --- Step 4: Compare the Results ---
println("--- OPF Solution Comparison: DC vs. AC ---")
println("Total DC OPF Cost: \$", round(result_dc["objective"], digits=2), "/hr")
println("Total AC OPF Cost: \$", round(result_ac["objective"], digits=2), "/hr")

println("\n" * "="^60)
println("           GENERATOR DISPATCH COMPARISON (MW)")
println("="^60)
println("Gen ID | Bus | DC Dispatch (MW) | AC Dispatch (MW) | Difference (MW)")
println("-----------------------------------------------------------------")

for (i, gen_dc) in result_dc["solution"]["gen"]
    gen_ac = result_ac["solution"]["gen"][i]
    pg_dc = gen_dc["pg"] * 100
    pg_ac = gen_ac["pg"] * 100
    diff = pg_ac - pg_dc
    bus_num = gen_dc["gen_bus"]
    @printf "   %-3s | %3d | %16.2f | %16.2f | %15.2f\n" i bus_num pg_dc pg_ac diff
end

println("\nQuestion for your report: Why is the total cost of the AC OPF higher than the DC OPF? (Hint: Think about losses).")

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 12 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 12 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 20 from -360.0 to -60.0 deg.
[w

LoadError: UndefVarError: `DCMPowerModel` not defined in `Main`
Suggestion: check for spelling errors or missing imports.



---
### **Problem 2: From Economic Dispatch to Security-Constrained OPF**

This problem demonstrates the evolution of dispatch methods. We will use the 3-bus system from our lecture.

*   **System Data:** 3 buses, 2 generators, 1 load.
    *   Gen 1 (Bus 1): Cost = `10*P`, Pmax = 5.0 p.u.
    *   Gen 2 (Bus 3): Cost = `25*P`, Pmax = 5.0 p.u.
    *   Load (Bus 2): `P_D = 2.0` p.u.
    *   Line reactances: `x₁₂ = 0.1`, `x₁₃ = 0.4`, `x₂₃ = 0.2`.
    *   Line thermal limit: `P_flow_max = 1.2` p.u. for the line between Bus 2 and 3.

(a) **Economic Dispatch:** Ignoring all transmission constraints, what is the cheapest way to serve the 2.0 p.u. load? What is the total cost?

(b) **DC Power Flow Check:** For the dispatch from part (a), calculate the DC power flow on the line between Bus 2 and 3. Is the dispatch feasible (i.e., is the flow below the thermal limit)?

(c) **DC OPF:** Using `PowerModels.jl`, solve the DC OPF for this system. What is the new, secure dispatch and the new total cost? What is the "cost of congestion"?

In [ ]:
# --- Problem 2 Solution ---

# --- System Data ---
P_D2 = 2.0
Cost_G1 = 10.0; Cost_G2 = 25.0;
Pmax_G1 = 5.0; Pmax_G2 = 5.0;
x12 = 0.1; x13 = 0.4; x23 = 0.2;
P_flow_max_23 = 1.2;

# --- Part (a): Economic Dispatch ---
# To serve 2.0 p.u. of load, the cheapest method is to use the cheapest generator.
P_G1_ed = Cost_G1*Pmax_G1;
P_G2_ed = Cost_G2*Pmax_G2;
total_cost_ed = P_G1_ed+P_G2_ed;
println("--- Part (a): Economic Dispatch ---")
@printf "Dispatch: P_G1 = %.1f p.u., P_G2 = %.1f p.u.\n" P_G1_ed P_G2_ed
@printf "Total Cost = \$%.1f/hr\n" total_cost_ed

# --- Part (b): DC Power Flow Check ---
# We need to find the angles that result from this dispatch.
# P_inj = B' * δ => δ = inv(B') * P_inj
b12 = 1/x12; b13 = 1/x13; b23 = 1/x23;
B_prime = [b12+b23  -b23; -b23  b13+b23]
# Injections at non-slack buses [2, 3]
P_inj = [-P_D2, P_G2_ed]
delta_vec = ... # YOUR CODE HERE: inv(B_prime) * P_inj
delta2 = delta_vec[1]; delta3 = delta_vec[2];

# Now calculate the flow on line 2-3
P_flow_23 = (delta2 - delta3) / x23
is_feasible = abs(P_flow_23) <= P_flow_max_23

println("\n--- Part (b): Feasibility Check ---")
@printf "Flow on Line 2-3 is %.3f p.u. (Limit is %.1f p.u.)\n" P_flow_23 P_flow_max_23
println("Is the dispatch feasible? ", is_feasible)

# --- Part (c): DC OPF Solution ---
# Create the PowerModels case dictionary for this problem
case_p2 = Dict(...) # YOUR CODE HERE to define the case
# ... (refer to lecture notes for the structure)

# Solve the DC OPF
result_dc_opf = ... # YOUR CODE HERE: solve_dc_opf(...)
P_G1_opf = result_dc_opf["solution"]["gen"]["1"]["pg"]
P_G2_opf = result_dc_opf["solution"]["gen"]["2"]["pg"]
total_cost_opf = result_dc_opf["objective"]
cost_of_congestion = total_cost_opf - total_cost_ed

println("\n--- Part (c): DC OPF Secure Dispatch ---")
@printf "Secure Dispatch: P_G1 = %.2f p.u., P_G2 = %.2f p.u.\n" P_G1_opf P_G2_opf
@printf "New Total Cost = \$%.2f/hr\n" total_cost_opf
@printf "The Cost of Congestion is \$%.2f/hr\n" cost_of_congestion

--- Part (a): Economic Dispatch ---
Dispatch: P_G1 = 2.0 p.u., P_G2 = 0.0 p.u.
Total Cost = $20.0/hr

--- Part (b): Feasibility Check ---
Flow on Line 2-3 is -0.286 p.u. (Limit is 1.2 p.u.)
Is the dispatch feasible? true


LoadError: KeyError: key :shunt not found



---
### **Problem 3: The Bidding Game**

You are the owner of the generator at **Bus 11** in the IEEE 30-bus system. Your goal is to maximize your profit. We will use the same congested market case from the lecture, where the line between Bus 2 and Bus 4 (Branch 4) is derated.

(a) **Marginal Bidding:** First, let's assume you and all other generators bid at your "marginal cost" (the costs we added in the lecture). Run the AC OPF and determine the dispatch, the LMP at your bus, and your resulting profit (`Profit = Dispatch_MW * LMP`). This is your baseline.

(b) **Strategic Bidding:** Now, you will develop a new, strategic bid for your generator at Bus 11. Assume all other generators continue to bid their marginal cost. Your goal is to change your bid to increase your profit. Can you exercise market power? Try submitting a higher bid. Does your dispatch go down? Does your profit go up? Experiment and find a bid that improves your profit over the baseline.

**Guidance:**
*   You will start with the `market_case` from the lecture.
*   For part (a), you will run the AC OPF with the default linear costs.
*   For part (b), you will create a `deepcopy` of the market case and **only modify the `gencost` entry for your generator** (Gen ID "5" is at Bus 11). Then re-run the OPF and calculate your new profit.


In [ ]:
# --- Problem 3 Solution ---
Pkg.add("HiGHS")
using PowerModels, Ipopt, Printf

# Load the built-in 30-bus test case
case_file_path = joinpath(dirname(pathof(PowerModels)), "..", "test/data/matpower/case30.m")
case_data = PowerModels.parse_file(case_file_path)

println("--- System Reconnaissance: IEEE 30-Bus System ---")

# How many buses, generators, loads, and branches?
num_buses = length(case_data["bus"])
num_gens = length(case_data["gen"])
num_loads = length(case_data["load"])
num_branches = length(case_data["branch"])
println("\nThis system has:")
println("- ", num_buses, " buses")
println("- ", num_gens, " generators")
println("- ", num_loads, " loads")
println("- ", num_branches, " transmission lines/transformers")

# What is the total system load?
total_load_pd = sum(load["pd"] for (i,load) in case_data["load"])
total_load_qd = sum(load["qd"] for (i,load) in case_data["load"])
println("\nTotal System Load = ", total_load_pd * 100, " MW and ", total_load_qd * 100, " MVAR")

for g in 1:length(case_data["gen"])
  index=string(g)
  case_data["gen"][index]["pmax"]=1
end


# Where are the generators located and what are their capacities?
println("\n--- Your Assets: Generator Information ---")
println("Gen ID | Located at Bus | Pmax (MW) | Default Cost (\$/MWh)")
println("---------------------------------------------------------")
for (i, gen) in case_data["gen"]
    bus_num = gen["gen_bus"]
    pmax = gen["pmax"] * 100
    @printf "   %-3s | %14d | %9.1f \n" i bus_num pmax
end

# --- ISO Action: Create the Congested Market Case ---
market_case = deepcopy(case_data)

# Reduce the rating of Branch 4 to 60% of its original value to force congestion
branch_to_constrain = "4"
market_case["branch"][branch_to_constrain]["rate_a"] *= 0.6
println("\nISO Intel: The thermal limit of Branch ", branch_to_constrain, " has been reduced. Expect congestion.")

# --- Part (a): Baseline Profit with Marginal Bidding ---
# Solve the AC OPF with the default costs
result_base = solve_ac_opf(market_case, Ipopt.Optimizer)

# Extract LMPs and calculate your baseline profit
# ... (Use the LMP extraction code from the lecture) ...
# --- ISO Action: Create the Congested Market Case ---
market_case = deepcopy(case_data)

# Reduce the rating of Branch 4 to 60% of its original value to force congestion
branch_to_constrain = "4"
market_case["branch"][branch_to_constrain]["rate_a"] *= 0.6
println("\nISO Intel: The thermal limit of Branch ", branch_to_constrain, " has been reduced. Expect congestion.")

println("\n--- Part (a): Baseline Profit ---")
# Print your dispatch, the LMP at Bus 11, and your total profit
println("\nTotal revenue (hourly) across all gens (congested LMPs) = \$", round(total_revenue, digits=2))


# --- Part (b): Strategic Bidding ---
# Create a copy of the market case to modify your bid
case_strategic = deepcopy(market_case)

# Define your new, strategic bid for Gen 5 (at Bus 11)
# Try increasing the cost. Let's try bidding $60/MWh instead of the original $50/MWh
my_strategic_bid = [60.0, 0.0]
# ... YOUR CODE HERE to update the "gencost" for generator "5" in the 'case_strategic' dictionary ...

# Solve the OPF with your new bid
result_strategic = ... # YOUR CODE HERE: solve_ac_opf(...)

# Extract LMPs and calculate your NEW profit
# ... YOUR CODE HERE ...

println("\n--- Part (b): Strategic Bidding Results ---")
# Print your new dispatch, the new LMP at Bus 11, and your new total profit
# ... YOUR CODE HERE ...

for(int i = 0;i<10;i++){
  if(>>){
    ;
  }
}

println("\nCompare your new profit to the baseline. Did your strategy work?")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


[info | PowerModels]: removing 3 cost terms from generator 4: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 1: [52.1378, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 5: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 2: [113.51659999999998, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 6: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 3: Float64[]
--- System Reconnaissance: IEEE 30-Bus System ---

This system has:
- 30 buses
- 6 generators
- 21 loads
- 41 transmission lines/transformers

Total System Load = 283.40000000000003 MW and 126.19999999999997 MVAR

--- Your Assets: Generator Information ---
Gen ID | Located at Bus | Pmax (MW) | Default Cost ($/MWh)
---------------------------------------------------------
   4   |              8 |     100.0 
   1   |              1 |     100.0 
   5   |             11 |     100.0 
   2   |              2 |     100.0 
   6   |             13 |

LoadError: UndefVarError: `total_revenue` not defined in `Main`
Suggestion: check for spelling errors or missing imports.